In [1]:
import os
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, NUMERIC
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import nltk
from whoosh.index import open_dir

# Open the index directory
index = open_dir("index")

# Access the schema
schema = index.schema

# Print the schema fields
print("Schema Fields:")
for field_name, field_type in schema.items():
    print(f"- {field_name}: {field_type}")

Schema Fields:
- contents: TEXT(format=Positions(boost=1.0), scorable=True, stored=False, unique=None)
- docID: NUMERIC(format=Existence(boost=1.0), scorable=None, stored=True, unique=False)


In [2]:
# Open a searcher to read stored contents
with index.searcher() as searcher:
    for docnum, fields in enumerate(searcher.all_stored_fields()):
        print(f"Document {docnum}:")
        for field_name, field_value in fields.items():
            print(f"  {field_name}: {field_value}")


Document 0:
  docID: 1
Document 1:
  docID: 2
Document 2:
  docID: 3
Document 3:
  docID: 4
Document 4:
  docID: 5
Document 5:
  docID: 6
Document 6:
  docID: 7
Document 7:
  docID: 8
Document 8:
  docID: 9
Document 9:
  docID: 10
Document 10:
  docID: 11
Document 11:
  docID: 12
Document 12:
  docID: 13
Document 13:
  docID: 14
Document 14:
  docID: 15
Document 15:
  docID: 16
Document 16:
  docID: 17
Document 17:
  docID: 18
Document 18:
  docID: 19
Document 19:
  docID: 20
Document 20:
  docID: 21
Document 21:
  docID: 22
Document 22:
  docID: 23
Document 23:
  docID: 24
Document 24:
  docID: 25
Document 25:
  docID: 26
Document 26:
  docID: 27
Document 27:
  docID: 28
Document 28:
  docID: 29
Document 29:
  docID: 30
Document 30:
  docID: 31
Document 31:
  docID: 32
Document 32:
  docID: 33
Document 33:
  docID: 34
Document 34:
  docID: 35
Document 35:
  docID: 36
Document 36:
  docID: 37
Document 37:
  docID: 38
Document 38:
  docID: 39
Document 39:
  docID: 40
Document 40:
  docI

In [3]:
from whoosh.fields import Schema, NUMERIC, TEXT

# Schema with a 'contents' field to store n-grams
schema = Schema(
    docID=NUMERIC(stored=True),
    contents=TEXT(stored=True)  # Store and index the preprocessed n-grams
)


In [4]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

def generate_ngrams(text, n):
    """
    Generate n-grams from the input text.
    """
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    ngram_list = [' '.join(gram) for gram in ngrams(tokens, n)]  # Create n-grams
    return ' '.join(ngram_list)  # Join n-grams into a single string


In [5]:
text = "This is an example document"
bigrams = generate_ngrams(text, 2)  # Create bigrams
print(bigrams)  # Output: "this is is an an example example document"


this is is an an example example document


In [6]:
from whoosh.index import create_in
import os

# Create the index directory
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")
ix = create_in("indexdir", schema)

# Write documents to the index
writer = ix.writer()

# Preprocess the document to generate bigrams
docID = 1
text = "This is an example document"
preprocessed_text = generate_ngrams(text, 2)  # Generate bigrams

# Add document to the index
writer.add_document(docID=docID, contents=preprocessed_text)
writer.commit()


In [7]:
from whoosh.qparser import QueryParser

# Open the index and create a searcher
with ix.searcher() as searcher:
    # Preprocess the query to generate bigrams
    query_text = generate_ngrams("example document", 2)
    parser = QueryParser("contents", schema=ix.schema)
    query = parser.parse(query_text)

    # Search the index
    results = searcher.search(query)
    for result in results:
        print(f"DocID: {result['docID']}, Contents: {result['contents']}")


DocID: 1, Contents: this is is an an example example document


In [4]:
with open('./doc/document.text', 'r') as f:
    text = f.read()
    docs = text.split('********************************************\n')[:-1]
    for doc in docs[0:3]:
        br = doc.find('Document')
        print(br)

0
0
0
